In [1]:
import time
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

In [2]:
start_time = time.time()

In [3]:
model_name = "distilgpt2"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=model_dtype,  # ✅ FIXED: was torch_dtype
    low_cpu_mem_usage=True
)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [11]:
model = model.to(device)
print(f"✅ Model on {device}")

✅ Model on cpu


In [16]:
pipe = pipeline(
    "text-generation",
    model="distilgpt2",
    torch_dtype=torch.float32,  # Legacy parameter for safety
    device=-1,  # Force CPU
    max_length=40,
    do_sample=True,
    temperature=0.7
)

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu


In [17]:
llm = HuggingFacePipeline(pipeline=pipe)

In [18]:

response = llm("Hello, AI!")
print(f"🤖: {response[:50]}...")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🤖: Hello, AI!‌ The last of the previous games from th...


In [21]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Create sophisticated prompt template
prompt_template = """You are a helpful AI assistant specialized in technical explanations.

Instructions:
- Keep answers clear and concise (2-3 sentences maximum)
- Use simple language suitable for beginners
- Focus on core concepts, avoid jargon

User Question: {question}

Assistant:"""

# Define the prompt template
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["question"]
)

print("📝 Prompt template created:")
print(f"Template preview: {prompt_template[:80]}...")
print(f"Input variables: {PROMPT.input_variables}")

# Create LangChain
print("\n🔧 Building LLMChain...")
chain = LLMChain(
    llm=llm,
    prompt=PROMPT,
    verbose=True  # Show chain execution for demo
)

print("✅ LangChain integration complete!")
print(f"Chain components: LLM + {len(PROMPT.input_variables)} prompt variables")

📝 Prompt template created:
Template preview: You are a helpful AI assistant specialized in technical explanations.

Instructi...
Input variables: ['question']

🔧 Building LLMChain...
✅ LangChain integration complete!
Chain components: LLM + 1 prompt variables


/Users/liutianxing/anaconda3/envs/llm_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
